In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
size = 200000

2023-08-19 03:27:33.108167: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 03:27:33.268684: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 03:27:33.269843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 03:27:34.160506: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_url = '/home/thinker/NLP/NLP/FromScratch/RNN_LSTM_GRU/error detection/data/data/train_data.csv'
test_url = '/home/thinker/NLP/NLP/FromScratch/RNN_LSTM_GRU/error detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/home/thinker/NLP/NLP/FromScratch/RNN_LSTM_GRU/error detection/data/stopwords_bangla.xlsx',index_col=False)

In [4]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [5]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [20]:
from sklearn.model_selection import train_test_split
MAX_FEATURES = 6000
EMBED_SIZE = 128
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(df_train['Comment'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['Comment'])
list_tokenized_test = tokenizer.texts_to_sequences(df_test['Comment'])
RNN_CELL_SIZE = 32
MAX_LEN = 60   # Since our mean length is 56.6
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN)
X_test = pad_sequences(list_tokenized_test,maxlen=MAX_LEN)
y_train = df_train['Error']
y_test = df_test['Error']

In [12]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [13]:
sequence_input = Input(shape=(MAX_LEN,), dtype="int32")
embedded_sequences = Embedding(MAX_FEATURES, EMBED_SIZE)(sequence_input)

In [14]:
lstm = Bidirectional(LSTM(RNN_CELL_SIZE, return_sequences = True), name="bi_lstm_0")(embedded_sequences)# Getting our LSTM outputs
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(RNN_CELL_SIZE, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)

In [15]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
context_vector, attention_weights = Attention(10)(lstm, state_h)
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.05)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = keras.Model(inputs=sequence_input, outputs=output)

In [16]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

In [17]:
BATCH_SIZE = 100
EPOCHS = 5
history = model.fit(X_train,y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.2)

Epoch 1/5
65/65 [==============================] - 10s 76ms/step - loss: 0.6827 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 3698.0000 - fn: 2730.0000 - accuracy: 0.5753 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5114 - val_loss: 0.6735 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 953.0000 - val_fn: 655.0000 - val_accuracy: 0.5927 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6181
Epoch 2/5
65/65 [==============================] - 4s 54ms/step - loss: 0.6360 - tp: 637.0000 - fp: 256.0000 - tn: 3442.0000 - fn: 2093.0000 - accuracy: 0.6346 - precision: 0.7133 - recall: 0.2333 - auc: 0.6625 - val_loss: 0.6090 - val_tp: 229.0000 - val_fp: 98.0000 - val_tn: 855.0000 - val_fn: 426.0000 - val_accuracy: 0.6741 - val_precision: 0.7003 - val_recall: 0.3496 - val_auc: 0.6924
Epoch 3/5
65/65 [==============================] - 4s 58ms/step - loss: 0.4434 - tp: 1901.0000 - fp: 414.0000 - tn: 3284.0000 - fn: 829.0000 - accuracy: 0.8066 - precision: 0.8212 - recall: 0.

In [18]:
pred = (model.predict(X_test) > 0.5).astype("int32")

63/63 [==============================] - 2s 8ms/step


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names = ['Correct','Incorrect']))

              precision    recall  f1-score   support

     Correct       0.68      0.78      0.73      1167
   Incorrect       0.62      0.48      0.54       844

    accuracy                           0.66      2011
   macro avg       0.65      0.63      0.63      2011
weighted avg       0.65      0.66      0.65      2011

